In [13]:
from time import time
import scipy.io as sio
import numpy as np
import json  
from math import sqrt   
from SSA import SSA_H_Plus
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from termcolor import colored
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16,preprocess_input
import keras.preprocessing as preprocessing
from keras.models import Model
from keras.utils import load_img 
import numpy as np
import tensorflow as tf  
import saliency.core as saliency
from IPython.display import HTML, display
import tabulate
import PIL.Image 
import cv2
import pandas as pd


In [11]:
class FeatureExtractor:
    def __init__(self) :
        base_model=VGG16(weights='vgg16_weights.h5') 
        self.model=Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    def extract(self,img_path):
        img = load_img(img_path, target_size=(224, 224)) 
        img=img.convert('RGB')
        x=preprocessing.image.image_utils.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feature=self.model.predict(x)[0]
        return feature/np.linalg.norm(feature)
    def extract_by_array(self,img_array): 
        x=np.expand_dims(img_array,axis=0)
        x=preprocess_input(x)
        feature=self.model.predict(x)[0]
        return feature/np.linalg.norm(feature)

In [12]:
class Saliency:
    def __init__(self) :
        m = VGG16(weights='vgg16_weights.h5')
        conv_layer = m.get_layer('block5_pool')
        self.model = Model([m.inputs], [conv_layer.output, m.output])
        
        self.class_idx_str = 'class_idx_str'
    def LoadImage(self,file_path):
        im = Image.open(file_path)
        im = im.resize((224,224))
        im = np.asarray(im)
        return im
    def PreprocessImage(self,im):
        im = preprocess_input(im)
        return im
    def call_model_function(self,images, call_model_args=None, expected_keys=None):
        target_class_idx =  call_model_args[self.class_idx_str]
        images = tf.convert_to_tensor(images)
        with tf.GradientTape() as tape:
            if expected_keys==[saliency.base.INPUT_OUTPUT_GRADIENTS]:
                tape.watch(images)
                _, output_layer = self.model(images)
                output_layer = output_layer[:,target_class_idx]
                gradients = np.array(tape.gradient(output_layer, images))
                return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
            else:
                conv_layer, output_layer = self.model(images)
                gradients = np.array(tape.gradient(output_layer, conv_layer))
                return {saliency.base.CONVOLUTION_LAYER_VALUES: conv_layer,
                        saliency.base.CONVOLUTION_OUTPUT_GRADIENTS: gradients}
    
    def GetSaliency(self,input,trs):
        im_orig = self.LoadImage(input)
        im = self.PreprocessImage(im_orig)
        _, predictions = self.model(np.array([im]))
        prediction_class = np.argmax(predictions[0])
        call_model_args = {self.class_idx_str: prediction_class}
        xrai_object = saliency.XRAI()
        xrai_attributions = xrai_object.GetMask(im, self.call_model_function, call_model_args, batch_size=10)
        mask = xrai_attributions >= np.percentile(xrai_attributions, trs)
        im_mask = np.array(im_orig)
        im_mask[~mask] = 0
        # plt.imshow(mask)
        # plt.show()
        X_True=[]
        Y_True=[]

       


        for i in range(mask.shape[0]): 
            for j in range(mask.shape[1]):
                if(mask[i][j]):
                    X_True.append(i)
                    Y_True.append(j)
        X_True=list( dict.fromkeys(X_True) )
        Y_True=list(dict.fromkeys(Y_True))
        
        Xmin=min(X_True)
        XMax=max(X_True)
        Ymin=min(Y_True)
        YMax=max(Y_True)
        # im_mask = np.array(im_orig)
        # im_mask[~mask] = 0

        # im_mask=im_mask[Xmin:XMax,Ymin:YMax]  

        # return im_mask
        return im_orig[Xmin:XMax,Ymin:YMax]  


In [14]:
CelebA_Annotation=pd.read_csv('data-set/celeba/Clean/Identity_CelebA.csv')


In [31]:
def GetNameAndClass(imgPath): 
    img_path_str=str(imgPath)
    img_path_splitted= img_path_str.split("/")
    ImageName=img_path_splitted[len(img_path_splitted)-1].replace(' ','')
    
    ThisName=CelebA_Annotation[CelebA_Annotation['Image']==ImageName].head(1)
    return_value=[str(ThisName['Identity'].values[0]),str(ImageName)] 
    return return_value

In [32]:
img_path_dogs=[str(x) for x in sio.loadmat('cached-data/new/celeba/img_path_celeba.mat')['img_path_obj']]
SSA_features=[x for x in sio.loadmat('cached-data/new/celeba/ssa_features_celeba.mat')['ssa_features']]
img_names=[GetNameAndClass(x)[1] for x in img_path_dogs] 
img_folders=[GetNameAndClass(x)[0] for x in img_path_dogs]  
all_results=[]

In [33]:
fe=FeatureExtractor() 
slc=Saliency()

In [38]:
# split_point=int(len(img_names)*.1)
split_point=100
count_of_results=5
training, test = img_names[split_point:], img_names[:split_point] 

Non Saliency part:

In [39]:
count_of_true=0
count_of_false=0
count_of_all=0

AllData=[['input','output1','output2','output3','output4','output5']]
for input in test: 
    index_of_test=img_names.index(input)
    input_folder=img_folders[index_of_test] 
    thisrec=[input_folder] 
    print('input:')
    print(input_folder)
    # img = mpimg.imread(img_path_dogs[index_of_test].replace(' ',''))
    # imgplot = plt.imshow(img)
    # plt.show()
    input_ssa=SSA_features[index_of_test]
    score_of_trained=[]
    for trained in training:
        index_of_trained=img_names.index(trained)
        trained_ssa=SSA_features[index_of_trained]
        distance= sqrt( sum(np.multiply(trained_ssa - input_ssa,trained_ssa - input_ssa)))
        trained_folder=img_folders[index_of_trained]
        score_of_trained.append([distance,trained,trained_folder])
    score_of_trained.sort()
    score_of_trained=score_of_trained[:count_of_results] 
    count_of_this_true=0
    print('output:')
    for res in score_of_trained:  
        if(res[2]==input_folder):
            print(colored(res[2],'green'))
            count_of_true=count_of_true+1
        else:
            print(colored(res[2],'red'))
            count_of_false=count_of_false+1
        
        thisrec.append(res[2])
        count_of_all=count_of_all+1 
        # img=mpimg.imread(os.getcwd()+'/data-set/celeba/Clean/'+res[1])
        # imgplot=plt.imshow(img)
        # plt.show() 
    print('--------')
    AllData.append(thisrec)    
AllData.append([count_of_all,0,0,0,0,count_of_true])
print('Count Of True: '+str(count_of_true))
print('Count Of False: '+str(count_of_false))
print('Count Of All: '+str(count_of_all))
table = tabulate.tabulate(AllData, tablefmt='html')
table

input:
4818
output:
9586
3830
9641
6626
193
--------
input:
227
output:
193
1878
193
8865
3784
--------
input:
3658
output:
2168
9546
7932
9586
7453
--------
input:
6480
output:
3668
3668
2346
7229
6480
--------
input:
9701
output:
5703
249
3780
4469
5703
--------
input:
7118
output:
1558
132
2321
6617
7299
--------
input:
3784
output:
3784
3784
2896
2896
3784
--------
input:
3226
output:
5544
6031
7665
3871
5703
--------
input:
6515
output:
1785
3464
3464
1375
1375
--------
input:
4069
output:
227
2999
7932
2570
7499
--------
input:
5703
output:
5703
5703
5327
5703
5703
--------
input:
4518
output:
2390
2390
4518
3668
4518
--------
input:
2961
output:
7123
4078
6031
5544
3276
--------
input:
910
output:
203
249
55
3640
8666
--------
input:
7298
output:
7298
6894
3123
8308
227
--------
input:
9260
output:
9220
7123
6676
10171
6223
--------
input:
649
output:
4125
5670
9107
7765
7765
--------
input:
2390
output:
2390
3668
7385
2390
1055
--------
input:
1049
output:
1049
4841
7229
7734
2

input,output1,output2,output3,output4,output5
4818,9586,3830,9641,6626,193
227,193,1878,193,8865,3784
3658,2168,9546,7932,9586,7453
6480,3668,3668,2346,7229,6480
9701,5703,249,3780,4469,5703
7118,1558,132,2321,6617,7299
3784,3784,3784,2896,2896,3784
3226,5544,6031,7665,3871,5703
6515,1785,3464,3464,1375,1375
4069,227,2999,7932,2570,7499


Saliency Part:

In [40]:
count_of_true=0
count_of_false=0
count_of_all=0
AllData=[['input','output1','output2','output3','output4','output5']]

for input in test: 
    index_of_test=img_names.index(input)
    input_folder=img_folders[index_of_test] 
    salient=slc.GetSaliency(('data-set/celeba/Clean/'+input), 65)
    
    print('input:')
    print(input)
    print(input_folder) 

    # imgplot = plt.imshow(salient)
    arr2im = PIL.Image.fromarray(salient)
    salient = arr2im.resize((224,224))
    salient = np.asarray(salient)
    salient_feature=fe.extract_by_array(salient)
    input_ssa=SSA_H_Plus(salient_feature)
    thisrec=[input_folder]  
    # plt.show() 
    score_of_trained=[]
    for trained in training:
        index_of_trained=img_names.index(trained)
        trained_ssa=SSA_features[index_of_trained]
        distance= sqrt( sum(np.multiply(trained_ssa - input_ssa,trained_ssa - input_ssa)))
        trained_folder=img_folders[index_of_trained]
        score_of_trained.append([distance,trained,trained_folder])
    score_of_trained.sort()
    score_of_trained=score_of_trained[:count_of_results]
    new_rec={
        'input':input
        ,'class':input_folder,
        'output':[{'name':x[1],'class':x[2]} for x in score_of_trained]
    } 
    print('output:')
    for res in score_of_trained: 
        if(res[2]==input_folder):
            print(colored(res[2],'green'))
            count_of_true=count_of_true+1
        else:
            print(colored(res[2],'red'))
            count_of_false=count_of_false+1
        count_of_all=count_of_all+1
        thisrec.append(res[2])
        
        # img=mpimg.imread(os.getcwd()+'/data-set/dogs/images/Images/'+res[2]+'/'+res[1])
        # imgplot=plt.imshow(img)
        # plt.show() 
        print('----')
    AllData.append(thisrec)    
AllData.append([count_of_all,0,0,0,0,count_of_true])
print('Count Of True: '+str(count_of_true))
print('Count Of False: '+str(count_of_false))
print('Count Of All: '+str(count_of_all))
table = tabulate.tabulate(AllData, tablefmt='html')
table

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/saliency/core/xrai.py:126: FutureWarning: `selem` is a deprecated argument name for `dilation`. It will be removed in version 1.0. Please use `footprint` instead.
  masks = [dilation(mask, selem=selem) for mask in masks]


input:
165523.png
4818


2022-11-28 17:53:33.684861: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 1s/step
output:
9641
----
2392
----
193
----
6626
----
3830
----
input:
025113.png
227
1/1 [==============================] - 0s 281ms/step
output:
193
----
1878
----
193
----
8865
----
3784
----
input:
138624.png
3658
1/1 [==============================] - 0s 214ms/step
output:
9586
----
5081
----
7932
----
4632
----
9586
----
input:
065245.png
6480
1/1 [==============================] - 0s 291ms/step
output:
3668
----
3668
----
2346
----
946
----
8308
----
input:
059594.png
9701
1/1 [==============================] - 0s 238ms/step
output:
5703
----
3780
----
249
----
3871
----
4286
----
input:
070660.png
7118
1/1 [==============================] - 0s 269ms/step
output:
1558
----
6617
----
8536
----
132
----
1049
----
input:
003856.png
3784
1/1 [==============================] - 0s 189ms/step
output:
3784
----
3784
----
2896
----
2896
----
3784
----
input:
076563.png
3226
1/1 [==============================] - 0s 139ms/step
output:
5703
----
5

KeyboardInterrupt: 